In [24]:
import time
from jassair.utils import get_dataset_path, Datasets
import cv2 as cv
from IPython.display import display_png, Image, clear_output

## Prepare test images from captured images

In [3]:
target_x = 640
data_path = get_dataset_path(Datasets.REAL_LIFE_TEST)

target_dir = data_path / "images"
target_dir.mkdir(exist_ok=True)

for i, image_path in enumerate(data_path.glob("*.jpg"), 1):
    image = cv.imread(image_path)
    
    h, w = image.shape[:2]
    h_2, w_2 = h // 2, w // 2
    x = min(h, w)
    x_2 = x // 2
    
    if x < target_x:
        raise ValueError(f"Image {image_path} is too small!, {image.shape[:2]}")
    
    h_start = h_2 - x_2
    h_stop = h_2 + x_2
    w_start = w_2 - x_2
    w_stop = w_2 + x_2        
    
    image = image[h_start:h_stop, w_start:w_stop]

    image = cv.resize(image, (target_x, target_x))
    cv.imwrite(f"{target_dir}/image_{i}.png", image) 

## Label the data

In [32]:
LABELS = ['Eichel 10', 'Eichel 6', 'Eichel 7', 'Eichel 8', 'Eichel 9', 'Eichel Ass', 'Eichel Konig', 'Eichel Ober', 'Eichel Under', 'Rose 10', 'Rose 6', 'Rose 7', 'Rose 8', 'Rose 9', 'Rose Ass', 'Rose Konig', 'Rose Ober', 'Rose Under', 'Schelle 10', 'Schelle 6', 'Schelle 7', 'Schelle 8', 'Schelle 9', 'Schelle Ass', 'Schelle Konig', 'Schelle Ober', 'Schelle Under', 'Schilte 10', 'Schilte 6', 'Schilte 7', 'Schilte 8', 'Schilte 9', 'Schilte Ass', 'Schilte Konig', 'Schilte Ober', 'Schilte Under']

LABELS = [f"{k}={v}" for k, v in enumerate(LABELS)]
LABELS = [" ".join(LABELS[x:x+4]) for x in range(0, len(LABELS), 4)]
LABELS = "\n".join(LABELS)

In [ ]:
image_dir = target_dir
label_dir = data_path / "labels"
label_dir.mkdir(exist_ok=True)

for i, image_path in enumerate(image_dir.glob("*.png"), 1):
    time.sleep(1)
    display_png(Image(image_path))
    time.sleep(3)
    a = input(f"What is this image?\n{LABELS}")
    with (label_dir / f"image_{i}.txt").open("w", encoding="utf-8") as f:
        f.write(str(a))
    clear_output()